In [181]:
import nltk
from nltk.corpus import brown
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
import re
from collections import Counter
from sklearn.model_selection import train_test_split
import math


Importing the corpus

In [182]:
nltk.download('brown')

sentences = [' '.join(words) for words in brown.sents()]

print(sentences[:5])  


[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\Aymane\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!


["The Fulton County Grand Jury said Friday an investigation of Atlanta's recent primary election produced `` no evidence '' that any irregularities took place .", "The jury further said in term-end presentments that the City Executive Committee , which had over-all charge of the election , `` deserves the praise and thanks of the City of Atlanta '' for the manner in which the election was conducted .", "The September-October term jury had been charged by Fulton Superior Court Judge Durwood Pye to investigate reports of possible `` irregularities '' in the hard-fought primary which was won by Mayor-nominate Ivan Allen Jr. .", "`` Only a relative handful of such reports was received '' , the jury said , `` considering the widespread interest in the election , the number of voters and the size of this city '' .", "The jury said it did find that many of Georgia's registration and election laws `` are outmoded or inadequate and often ambiguous '' ."]


__Corpus Normalization Steps__

Before processing any corpus, several steps must be followed to prepare the corpus in a format suitable for analysis. This process is called **normalization**. The steps include:

__Case Folding__
- Transform the entire corpus content into **lowercase characters** to ensure uniformity.

__Tokenization__
- Divide the text into smaller units, such as:
  - **Words**
  - **Subwords**
  - **Other entities**
- The choice of units depends on various factors, including the algorithm used and the corpus characteristics.

__Special Characters Management__
- Remove all **punctuation marks**, as I do not consider them as words in this application.
- Add two special characters:
  - `<s>`: Marks the **start** of a sentence.
  - `</s>`: Marks the **end** of a sentence.


In [183]:
lower_sentences = [sentence.lower() for sentence in sentences]

print(lower_sentences[:5])  


["the fulton county grand jury said friday an investigation of atlanta's recent primary election produced `` no evidence '' that any irregularities took place .", "the jury further said in term-end presentments that the city executive committee , which had over-all charge of the election , `` deserves the praise and thanks of the city of atlanta '' for the manner in which the election was conducted .", "the september-october term jury had been charged by fulton superior court judge durwood pye to investigate reports of possible `` irregularities '' in the hard-fought primary which was won by mayor-nominate ivan allen jr. .", "`` only a relative handful of such reports was received '' , the jury said , `` considering the widespread interest in the election , the number of voters and the size of this city '' .", "the jury said it did find that many of georgia's registration and election laws `` are outmoded or inadequate and often ambiguous '' ."]


Now I will use regular expressions to match all the non alphanumerical characters and remove them (except the white space)

In [184]:
cleaned_sentences = [re.sub(r'[^\w\s]',' ',sentence) for sentence in lower_sentences]
print(cleaned_sentences[:5]) 

['the fulton county grand jury said friday an investigation of atlanta s recent primary election produced    no evidence    that any irregularities took place  ', 'the jury further said in term end presentments that the city executive committee   which had over all charge of the election      deserves the praise and thanks of the city of atlanta    for the manner in which the election was conducted  ', 'the september october term jury had been charged by fulton superior court judge durwood pye to investigate reports of possible    irregularities    in the hard fought primary which was won by mayor nominate ivan allen jr   ', '   only a relative handful of such reports was received      the jury said      considering the widespread interest in the election   the number of voters and the size of this city     ', 'the jury said it did find that many of georgia s registration and election laws    are outmoded or inadequate and often ambiguous     ']


For the tokenization task, I will use the Punkt tokenizer, a prebuilt and pretrained tokenizer from NLTK. Later, I plan to implement the BPE (Byte Pair Encoding) algorithm, which is another tokenization approach, to demonstrate the steps behind it. For now, I will stick with the Punkt tokenizer.

In [185]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt_tab')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Aymane\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Aymane\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Aymane\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Aymane\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [186]:
tokenized_sentences = [word_tokenize(sentence)for sentence in cleaned_sentences]
print(tokenized_sentences[:5])

[['the', 'fulton', 'county', 'grand', 'jury', 'said', 'friday', 'an', 'investigation', 'of', 'atlanta', 's', 'recent', 'primary', 'election', 'produced', 'no', 'evidence', 'that', 'any', 'irregularities', 'took', 'place'], ['the', 'jury', 'further', 'said', 'in', 'term', 'end', 'presentments', 'that', 'the', 'city', 'executive', 'committee', 'which', 'had', 'over', 'all', 'charge', 'of', 'the', 'election', 'deserves', 'the', 'praise', 'and', 'thanks', 'of', 'the', 'city', 'of', 'atlanta', 'for', 'the', 'manner', 'in', 'which', 'the', 'election', 'was', 'conducted'], ['the', 'september', 'october', 'term', 'jury', 'had', 'been', 'charged', 'by', 'fulton', 'superior', 'court', 'judge', 'durwood', 'pye', 'to', 'investigate', 'reports', 'of', 'possible', 'irregularities', 'in', 'the', 'hard', 'fought', 'primary', 'which', 'was', 'won', 'by', 'mayor', 'nominate', 'ivan', 'allen', 'jr'], ['only', 'a', 'relative', 'handful', 'of', 'such', 'reports', 'was', 'received', 'the', 'jury', 'said', '

Now, I add special markers to the beginning and end of each sentence. Note that I add two `<s>` markers at the beginning of each sentence because I will be using a trigram model.

In [187]:
normalized_sentences = [['<s>','<s>'] + sentence + ['</s>'] for sentence in tokenized_sentences]
print(normalized_sentences[:5])

[['<s>', '<s>', 'the', 'fulton', 'county', 'grand', 'jury', 'said', 'friday', 'an', 'investigation', 'of', 'atlanta', 's', 'recent', 'primary', 'election', 'produced', 'no', 'evidence', 'that', 'any', 'irregularities', 'took', 'place', '</s>'], ['<s>', '<s>', 'the', 'jury', 'further', 'said', 'in', 'term', 'end', 'presentments', 'that', 'the', 'city', 'executive', 'committee', 'which', 'had', 'over', 'all', 'charge', 'of', 'the', 'election', 'deserves', 'the', 'praise', 'and', 'thanks', 'of', 'the', 'city', 'of', 'atlanta', 'for', 'the', 'manner', 'in', 'which', 'the', 'election', 'was', 'conducted', '</s>'], ['<s>', '<s>', 'the', 'september', 'october', 'term', 'jury', 'had', 'been', 'charged', 'by', 'fulton', 'superior', 'court', 'judge', 'durwood', 'pye', 'to', 'investigate', 'reports', 'of', 'possible', 'irregularities', 'in', 'the', 'hard', 'fought', 'primary', 'which', 'was', 'won', 'by', 'mayor', 'nominate', 'ivan', 'allen', 'jr', '</s>'], ['<s>', '<s>', 'only', 'a', 'relative',

In [192]:
def trigram_probability(trigram, trigrams_count, bigrams_count):
    trigram_count = trigrams_count.get(trigram, 0)
    bigram = (trigram[0], trigram[1])
    bigram_count = bigrams_count.get(bigram, 0)
    if bigram_count == 0:
        return 0
    probability = trigram_count / bigram_count
    return probability
def bigram_probability(bigram, bigrams_count, unigrams_count):
    bigram_count = bigrams_count.get(bigram, 0)
    unigram = bigram[0]
    unigram_count = unigrams_count.get(unigram, 0)
    if unigram_count == 0:
        return 0
    probability = bigram_count / unigram_count
    return probability
def unigram_probability(word, unigrams_count):
    word_count = unigrams_count.get(word, 0)
    total_count = sum(unigrams_count.values())
    if total_count == 0:
        return 0
    probability = word_count / total_count
    return probability


Now that I have defined the n-gram probabilities, it is time to implement interpolation smoothing to achieve a better probability distribution. This approach is preferable to the Maximum Likelihood Estimate, which cannot effectively handle zero-probability n-grams.

In [193]:
def static_interpolation(trigram, trigrams_count, bigrams_count, unigrams_count, t, b, u, z, vocab_size):
    bigram = (trigram[1], trigram[2])
    word = trigram[2]  
    
    p_trigram = trigram_probability(trigram, trigrams_count, bigrams_count)
    
    p_bigram = bigram_probability(bigram, bigrams_count, unigrams_count)
    
    p_unigram = unigram_probability(word, unigrams_count)
    
    p_zerogram = 1 / vocab_size
    
    probability = t * p_trigram + b * p_bigram + u * p_unigram + z * p_zerogram
    
    return probability


Now, I divide the corpus into two sets: a training set and a test set. The training set is used to calculate the probabilities of the n-grams. Here, I will compute all the trigrams, bigrams, and unigrams in the corpus. These will be used for interpolation smoothing to handle zero-probability n-grams and achieve better performance.

In [194]:
train_corpus, test_corpus = train_test_split(normalized_sentences, test_size=0.1, random_state=42)
train_unigrams = [word for sentence in train_corpus for word in sentence]
training_vocab = set(unigrams_count.keys())
vocab_size = len(training_vocab)
train_bigrams = [bigram for sentence in train_corpus for bigram in ngrams(sentence, 2)]
train_trigrams = [trigram for sentence in train_corpus for trigram in ngrams(sentence, 3)]
train_unigrams = [word for sentence in train_corpus for word in sentence]

bigrams_count = Counter(train_bigrams)
trigrams_count = Counter(train_trigrams)
unigrams_count = Counter(train_unigrams)

test_trigrams = [trigram for sentence in test_corpus for trigram in ngrams(sentence, 3)]

Evaluating a language model is a challenging task. There are two plausible approaches:  

Extrinsic Evaluation: This involves assessing the model based on its performance in the specific task it is designed for. In this case, the evaluation focuses on the quality of its predictions.  

Intrinsic Evaluation: This approach evaluates the model independently of its specific task. One of the most commonly used metrics for intrinsic evaluation is Perplexity, which measures how 'confused' the model is when predicting the next word.

In [195]:
def calculate_perplexity(test_trigrams, trigrams_count, bigrams_count, unigrams_count, t, b, u, z, vocab_size):
    log_prob_sum = 0
    total_trigrams = len(test_trigrams)

    for trigram in test_trigrams:
        prob = static_interpolation(trigram, trigrams_count, bigrams_count, unigrams_count, t, b, u, z, vocab_size)
        
        if prob > 0:
            log_prob_sum += math.log2(prob)
        else:
            log_prob_sum += float('-inf')  

    perplexity = 2 ** (-log_prob_sum / total_trigrams)
    return perplexity


The choice of weights has a significant impact on the performance of the model. There are several methods to determine these parameters, such as the Expectation-Maximization algorithm, which iteratively converges to optimal values. However, to keep it simple, I will try various values and select the ones that perform well.

In [196]:
t, b, u, z = 0.5, 0.3, 0.15, 0.05 

perplexity = calculate_perplexity(test_trigrams, trigrams_count, bigrams_count, unigrams_count, t, b, u, z, vocab_size)
print(f"Perplexity: {perplexity}")


Perplexity: 719.3679912031918


In [197]:
t, b, u, z = 0.3, 0.5, 0.1, 0.1  

perplexity = calculate_perplexity(test_trigrams, trigrams_count, bigrams_count, unigrams_count, t, b, u, z, vocab_size)
print(f"Perplexity: {perplexity}")


Perplexity: 663.2839835853889


Now let's try an application of this model which is the prediction of the next word. This is used in many applications such as Gmail, when receiving a message or when typing a message the next word is predicted .

In [198]:
def suggest_next_word(context, trigrams_count, bigrams_count, unigrams_count, t, b, u, z, vocab_size):
    if len(context) == 2:
        w1, w2 = context
    elif len(context) == 1:
        w1, w2 = '<s>', context[0]
    else:
        w1, w2 = '<s>', '<s>'

    candidates = {}
    for word in unigrams_count.keys():
        if word == '</s>' or word == '<s>' :  
            continue
        trigram = (w1, w2, word)
        prob = static_interpolation(trigram, trigrams_count, bigrams_count, unigrams_count, t, b, u, z, vocab_size)
        candidates[word] = prob

    best_word = max(candidates, key=candidates.get)
    return best_word


Since we have evaluated the model, we can now train it on the whole dataset

In [199]:
train_corpus =  train_corpus + test_corpus 

train_unigrams = [word for sentence in train_corpus for word in sentence]
training_vocab = set(unigrams_count.keys())
vocab_size = len(training_vocab)
train_bigrams = [bigram for sentence in train_corpus for bigram in ngrams(sentence, 2)]
train_trigrams = [trigram for sentence in train_corpus for trigram in ngrams(sentence, 3)]
train_unigrams = [word for sentence in train_corpus for word in sentence]

bigrams_count = Counter(train_bigrams)
trigrams_count = Counter(train_trigrams)
unigrams_count = Counter(train_unigrams)

test_trigrams = [trigram for sentence in test_corpus for trigram in ngrams(sentence, 3)]
t, b, u, z = 0.3, 0.5, 0.1, 0.1

We choose a context, which means in our case a sequence of two words, and the third word is predicted by the model.

In [200]:
context = ['i','love']
suggest_next_word(context, trigrams_count, bigrams_count, unigrams_count, t, b, u, z, vocab_size)


'you'

In [201]:
context = ['he','hates']
suggest_next_word(context, trigrams_count, bigrams_count, unigrams_count, t, b, u, z, vocab_size)

'that'

In [202]:
context = ['i','eat']
suggest_next_word(context, trigrams_count, bigrams_count, unigrams_count, t, b, u, z, vocab_size)

'too'

In [203]:
context = ['the', 'president']
suggest_next_word(context, trigrams_count, bigrams_count, unigrams_count, t, b, u, z, vocab_size)

'of'

In [204]:
context = ['are', 'the']
suggest_next_word(context, trigrams_count, bigrams_count, unigrams_count, t, b, u, z, vocab_size)

'most'

In [205]:
context = ['thank', 'you']
suggest_next_word(context, trigrams_count, bigrams_count, unigrams_count, t, b, u, z, vocab_size)

'for'

In [206]:
context = ['the', 'cat']
suggest_next_word(context, trigrams_count, bigrams_count, unigrams_count, t, b, u, z, vocab_size)

'and'

In [207]:
context = ['president', 'of']
suggest_next_word(context, trigrams_count, bigrams_count, unigrams_count, t, b, u, z, vocab_size)

'the'

In [208]:
context = ['of', 'the']
suggest_next_word(context, trigrams_count, bigrams_count, unigrams_count, t, b, u, z, vocab_size)

'united'

In [209]:
context = ['the', 'united']
suggest_next_word(context, trigrams_count, bigrams_count, unigrams_count, t, b, u, z, vocab_size)

'states'

In [210]:
context = ['the','other']
suggest_next_word(context, trigrams_count, bigrams_count, unigrams_count, t, b, u, z, vocab_size)

'hand'

The predicted words are logical, the task is done correctly :D  